# 라이브러리 불러오기

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
import seaborn as sns
import os
import tensorflow as tf
from tensorflow import keras
from PIL import Image

In [ ]:
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization, Lambda

# 데이터 불러오기

깃허브에서 가져오기(구글 코랩 환경)

In [ ]:
os.chdir('../home')
os.getcwd()

'/home'

In [ ]:
!git clone https://github.com/HCturtle/pokemon-type.git

Cloning into 'pokemon-type'...
remote: Enumerating objects: 880, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 880 (delta 0), reused 0 (delta 0), pack-reused 874
Receiving objects: 100% (880/880), 2.38 MiB | 18.62 MiB/s, done.
Resolving deltas: 100% (11/11), done.


타입이 적혀있는 csv 파일 가져오기

In [ ]:
csv_data = pd.read_csv("./pokemon-type/data/pokemon.csv")
csv_data.info

<bound method DataFrame.info of             Name     Type1   Type2
0      bulbasaur     Grass  Poison
1        ivysaur     Grass  Poison
2       venusaur     Grass  Poison
3     charmander      Fire     NaN
4     charmeleon      Fire     NaN
..           ...       ...     ...
804    stakataka      Rock   Steel
805  blacephalon      Fire   Ghost
806      zeraora  Electric     NaN
807       meltan     Steel     NaN
808     melmetal     Steel     NaN

[809 rows x 3 columns]>

이미지 데이터 목록을 가져와 타입을 찾아서 pandas.DataFrame으로 만들기

In [ ]:
images_data = "./pokemon-type/data/images/"

In [ ]:
pokemon_filenames = os.listdir(images_data)

pnames, primary_types,secondary_types = [], [], []
for path in os.listdir(images_data):
        pokemon_name = path.split('.')[0] #'이름(영어버전).확장자' -> '이름(영어버전)'
        row = csv_data[csv_data['Name']==pokemon_name] #이름으로 해당 포켓몬 찾기
        type1 = csv_data['Type1'][int(row.index.values)]
        type2 = csv_data['Type2'][int(row.index.values)]
        pnames.append(pokemon_name.capitalize())#ex) abomasnow -> Abomasnow
        primary_types.append(type1)  
        secondary_types.append(type2)
#포켓몬 데이터 pandas.DataFrame 생성
pokemon_d = {'Image':pokemon_filenames, 'Pokemon':pnames, 'Primary_Type':primary_types, 'Secondary_Type':secondary_types}
pokemon_data = pd.DataFrame(pokemon_d)
pokemon_data['Primary_Type'].value_counts()

Water       114
Normal      105
Grass        78
Bug          72
Fire         53
Psychic      53
Rock         46
Electric     40
Poison       34
Ground       32
Fighting     29
Dark         29
Dragon       27
Ghost        27
Steel        26
Ice          23
Fairy        18
Flying        3
Name: Primary_Type, dtype: int64

In [ ]:
pokemon_data.head()

,Image,Pokemon,Primary_Type,Secondary_Type
0,volcarona.png,Volcarona,Bug,Fire
1,grovyle.png,Grovyle,Grass,NaN
2,gengar.png,Gengar,Ghost,Poison
3,florges.png,Florges,Fairy,NaN
4,spoink.png,Spoink,Psychic,NaN


In [ ]:
#포켓몬 수만큼 임의의 순서 가져오기
numbers = []
for i in range(1,csv_data.shape[0]+1):
    numbers.append(i)
csv_data['pkn'] = numbers
IMG_DIR = '/kaggle/input/pokemon-images-dataset/pokemon/pokemon'
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(IMG_DIR) if isfile(join(IMG_DIR, f))]

import re
dataframe_img = pd.DataFrame([])
images = []
pokemon_number = []
for img in onlyfiles:
    if not re.search('-', img):
        pkn = img.split('.')
        n = re.sub("[^0-9]", "", pkn[0])
        path = str(img)
        images.append(path)
        pokemon_number.append(n)
dataframe_img['Image'] = images
dataframe_img['pkn'] = pokemon_number
dataframe_img['pkn'] = dataframe_img['pkn'].astype(int)
pokemon_test = csv_data.merge(dataframe_img, left_on='pkn', right_on='pkn')
pokemon_test.rename(columns={'Name':'Pokemon','Type1':'Primary_Type','Type2':'Secondary_Type'},inplace=True)
pokemon_test['Primary_Type'].value_counts()

'Abomasnow'

디렉토리 초기화

In [ ]:
#디렉토리 삭제
import shutil
if os.path.exists('train/'):
    shutil.rmtree('train/') 
if os.path.exists('test/'):
    shutil.rmtree('test/') 
if os.path.exists('val/'):
    shutil.rmtree('val/') 

#디렉토리 생성
from shutil import copyfile, copy2
if not os.path.exists('train/'):
    os.mkdir('train/')
if not os.path.exists('test/'):
    os.mkdir('test/')
if not os.path.exists('val/'):
    os.mkdir('val/')

#타입별 디렉토리 생성
for class_ in pokemon_data['Primary_Type'].unique():
    if not os.path.exists('train/'+str(class_)+'/'):
        os.mkdir('train/'+str(class_)+'/')
    if not os.path.exists('test/'+str(class_)+'/'):
        os.mkdir('test/'+str(class_)+'/')
    if not os.path.exists('val/'+str(class_)+'/'):
        os.mkdir('val/'+str(class_)+'/')